**Montar(Mount) Azure Data Lake para el Proyecto**

In [0]:
def mount_adls(storage_account_name, container_name):
    #Obtenemos el Secret Key de Key Vault
    client_id=dbutils.secrets.get(scope="movie-history-secret-scope",key="client-id")
    directory_id=dbutils.secrets.get(scope="movie-history-secret-scope",key="directory-id")
    client_secret=dbutils.secrets.get(scope="movie-history-secret-scope",key="client-secret")

    #Establecemos las configuraciones de Spark
    configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": client_secret,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{directory_id}/oauth2/token"}
    
    #Desmontar(unmount) el montaje(mount) si ya existe
    if any(mount.mountPoint == f"/mnt/{storage_account_name}/{container_name}" for mount in dbutils.fs.mounts()):
        dbutils.fs.unmount(f"/mnt/{storage_account_name}/{container_name}")

    #Establecer un Mount(Montar) el Contenedor del Storage Account
    dbutils.fs.mount(
    source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
    mount_point = f"/mnt/{storage_account_name}/{container_name}",    #mnt + nombre de cuenta de almacenamiento + nombre del contenedor
    extra_configs = configs) 

    #Listar los Mount
    display(dbutils.fs.mounts())

In [0]:
mount_adls("mymoviehistory", "silver")  #montar contener silver

In [0]:
mount_adls("mymoviehistory", "gold")  #montar contenedor gold.

####Montar(mount) el contenedor "bronze"

In [0]:
mount_adls("mymoviehistory", "bronze")  #nos ha creado el montaje bronze, poniendo en una sola celda todo el código y sin tener que indicar más que unas variables.

In [0]:
#display(dbutils.fs.ls("abfss://demo@mymoviehistory.dfs.core.windows.net"))   #nombre de contenedor + nombre cuenta de almacenamiento
display(dbutils.fs.ls("/mnt/mymoviehistory/demo"))   #nombre de contenedor + nombre cuenta de almacenamiento

In [0]:
display(spark.read.csv("/mnt/mymoviehistory/demo/movie.csv"))

####Montar(mount) el contenedor "demo"

In [0]:
mount_adls("mymoviehistory", "demo")  #nos ha creado el montaje bronze, poniendo en una sola celda todo el código y sin tener que indicar más que unas variables.